[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/joshmaglione/CS102-Jupyter/main?labpath=.%2FWeek04.ipynb) 

<a href="https://colab.research.google.com/github/joshmaglione/CS102-Jupyter/blob/main/Week04.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[View on GitHub](https://github.com/joshmaglione/CS102-Jupyter/blob/main/Week04.ipynb)

# Week 4: Universal functions and broadcasting

*Universal functions* (or UFuncs) are operations applied to `ndarray`. 

Operations are generally unary (takes one argument) or binary (takes two arguments).

## What do we mean by operations?

For example: $v = (2, -3, 0, 9)$.

Unary operation of $-$
$$
    -v = (-2, 3, 0, -9).
$$

Binary operation of $+$
$$
    v+v = (4, -6, 0, 18).
$$

NumPy has asserted that multiplication of `ndarray` are to be *component-wise*. 

There is no canonical choice of what multiplication should be, but this is a useful choice in practice. 

Continuing our example: $*$ is a binary operation
$$
    v * v = (3, 9, 0, 81). 
$$

Of course so is division, but this would not work for our specific $v$. 

## Time comparisons

Last week, we saw hints of NumPy being significantly faster than Python at running through elements of containers.

Apparently, this isn't always true, at least for Python 3.12.

In [ ]:
import numpy as np

Let's create large arrays (but still small compared to real-world examples). 

In [ ]:
N = 10^9
apy = range(1, N + 1)
bpy = range(2, N + 2)
anp = np.arange(1, N + 1)
bnp = np.arange(2, N + 2)

Let's multiply both vectors.

In [ ]:
%timeit _ = [apy[i] * bpy[i] for i in range(N)]

We will use a UFunc here.

In [ ]:
%timeit _ = anp * bnp

**Fun fact.** I ran the above cells on my MacBook Pro which has an [Apple M2](https://en.wikipedia.org/wiki/Apple_M2) chip, and the speeds are essentially the same! 

In [ ]:
apy = [np.random.randint(1, 100) for _ in range(N)]
bpy = [np.random.randint(1, 100) for _ in range(N)]
anp = np.random.randint(1, 100, size=N)
bnp = np.random.randint(1, 100, size=N)

In [ ]:
%timeit _ = [apy[i] * bpy[i] for i in range(N)]

In [ ]:
%timeit _ = anp * bnp

Although Python might perform on par with NumPy, it is safer to just use NumPy if performance might be an issue.

## Ufuncs now, seriously

NumPy offers many *vectorized* alternatives to the usual operations.

The point is that this pushes the loop down into the compiled C code, resulting in a more efficient execution. 

NumPy has over [60 Ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html#available-ufuncs).

Ufuncs are easy to use: just apply the operation on arrays

In [ ]:
a = np.random.randint(1, 10, size=3)
b = np.random.randint(1, 10, size=3)
c = a + b
print(f"    array a = {a}")
print(f"    array b = {b}")
print(f"array a + b = {c}")

More of the usual mathematical operations

In [ ]:
print(a * b)

In [ ]:
print(a / b)

In [ ]:
print(a // b)

In [ ]:
print(2**a)         # this is the exponentiation operation.

In [ ]:
print(a**2)

In [ ]:
# print(a**(-1))      # Won't convert

If we want to change the data type (`dtype`) of an array, best to use `astype` method.

In [ ]:
a_fl = a.astype('f')
print(a_fl)
print(a_fl**(-1))

In [ ]:
print(a % 2)

Can do bitwise operations as well

In [ ]:
print(~a)

In [ ]:
print(a^a)

From the documentation:

> Some of these ufuncs are called automatically on arrays when the relevant infix notation is used (e.g., `add(a, b)` is called internally when `a + b` is written and `a` or `b` is an ndarray).
>
> Nevertheless, you may still want to use the **ufunc call** in order to use the optional output argument(s) to place the output(s) in an object (or objects) of your choice.

In [ ]:
print(a + b)
print(np.add(a, b))

The methods can prevent additional work. 

For example, take the example where we wanted to invert an array of ints.

We can use `np.power` and specify the output `dtype` all at once.

In [ ]:
np.power(a, -1, dtype='f')

Using the keyword argument `out`, one can specify the target output.

In [ ]:
z = np.zeros(20)
np.power(2, range(10), out=z[1::2])
print(z)

Often the methods are not required, but sometimes they can be exactly the right tool.

### Trigonometric functions (try it yourself)

In [ ]:
thetas = np.linspace(0, np.pi, 3)
print(thetas)

In [ ]:
print(f"sin(theta) = {np.sin(thetas)}")
print(f"cos(theta) = {np.cos(thetas)}")
print(f"tan(theta) = {np.tan(thetas)}")

One can perform the inverse trigonometric functions as well.

In [ ]:
x = [-1, 0, 1]
print(f"       x  = {x}")
print(f"arcsin(x) = {np.arcsin(x)}")
print(f"arccos(x) = {np.arccos(x)}")
print(f"arctan(x) = {np.arctan(x)}")

### Exponentials and logarithms (try it yourself)

We can take powers of $e$ with `np.exp`, of $2$ with `np.exp2`, or more general bases with `np.power`.

In [ ]:
x = [1, 2, 3]
print(f"   x = {x}")
print(f" e^x = {np.exp(x)}")
print(f" 2^x = {np.exp2(x)}")
print(f"pi^x = {np.power(np.pi, x)}")

The function `np.log` gives the natural logarithm, `np.log2` gives the base-2 logarithm, and `np.log10` gives the base-10 logarithm.

In [ ]:
x = [1, 2, 4, 10]
print(f"       x = {x}")
print(f"  log(x) = {np.log(x)}")
print(f" log2(x) = {np.log2(x)}")
print(f"log10(x) = {np.log10(x)}")

### Aggregates and Accumulations

Time for a quick journey down a rabbit hole

![](imgs/rabbit.jpg)

There is a Python module that ships with Python called `functools` (unofficially "functional tools").

In it there is a function called `reduce` which basically takes a list whose entries have a particular type $t$ and returns something of type $t$.

**Oversimplification!**

Can also think of this as *aggregating* the data.

You also need to tell `reduce` *how* to, well, reduce the list.

Two arguments:
1. a function,
2. a list,
3. (optional) a starting value.

In [ ]:
from functools import reduce

# Argument 1 : a function
def add(x, y):
    return x + y

# Argument 2 : a list
a = ["Hello", "world!"]

# Argument 3 : starting value
start = ""

We can reduce our list `a` to a string using the recipe given by `add`.

In [ ]:
reduce(add, a, start)

OK back to NumPy.

Often when one uses `reduce`, a standard operation is used to reduce the list to a value.

NumPy Ufuncs have a `reduce` method builtin, so instead of calling `reduce` from `functools`, one can use a likely better performing alternative.

In [ ]:
a = np.arange(10)
print(a)
np.add.reduce(a)

One should take `np.add.reduce(a)` as `reduce(add, a, 0)`.

One can set an initial value with `initial`.

In [ ]:
np.add.reduce(a, initial=10)

In [ ]:
np.multiply.reduce(a[1:])

Sometimes one is not *only* interested in the final aggregated value, but instead one wants all of accumulated values at every step.

Instead of `reduce` one should use `accumulate`.

In [ ]:
print(np.add.accumulate(a))

In [ ]:
print(np.multiply.accumulate(a[1:]))

## Ufuncs in higher dimensions

As before, going from $1$-dimensional arrays to $2$-dimensional arrays is sometimes the tricky step. From $2$-dimensions to $n$-dimensions, it is generally straight-forward.

Ufuncs can be applied to arrays of the same *shape*.

Above we were only looking at $1$-dimensional arrays with the same number of entries (i.e. shape).

In [ ]:
a = np.arange(20).reshape(4, 5)
b = np.random.randint(1, 10, size=(4, 5))
print(a)
print()
print(b)

In [ ]:
print(a + b)

In [ ]:
print(a * b)

**WARNING:** Be aware that `<matrix> * <matrix>` is **NOT** matrix multiplication. I make this mistake all the time 🤕

For that, use the `@` operator: `<matrix> @ <matrix>`

In [ ]:
# print(a @ b)      # Wrong shapes (4,5) x (4,5) is incompatible. 

In [ ]:
print(np.power(2, a))

You can imagine higher dimensions and all the other Ufuncs we described (and more).

## Broadcasting

Actually, I lied, but only because I knew I would make you aware of it right now.

You **CAN** use Ufuncs beween arrays of different *shapes* and *sizes*.

**Broadcasting** is a means of taking two arrays of different shapes and sizes and making them the same shape and size.

So really, I didn't lie: Ufuncs need arrays of the same shape and size. It's just that broadcasting gets built into the function as well,so it appears not to be necessary.

### 1-dimensional arrays

From [linear algebra](https://en.wikipedia.org/wiki/Linear_algebra), there is an operation with vectors called [*scalar multiplication*](https://en.wikipedia.org/wiki/Scalar_multiplication).

This can be viewed as an example of *broadcasting*.

In [ ]:
a = np.array([1, 2, 3])
b = np.array([2, 2, 2])
c = 2
print(a * b)                    # Ufunc *
print(a * c)                    # Broadcasting

Here is a visualization of broadcasting:

![](imgs/broadcasting_1.png)

**Note.** Taken directly from NumPy's documentation:
> The code in the second example is more efficient than that in the first because broadcasting moves less memory around during the multiplication

So if it makes sense to broadcast, do it.

#### Broadcasting rules

Suppose we have two arrays `a` and `b` of different shapes and we apply a Ufunc to them. Numpy appliesa simple check to verify if broadcasting can be done.

It runs the following check on each axis, starting from the *largest* indexed axis (i.e. rightmost). 

Arrays `a` and `b` are *compatible* along axis `i` if either
1. `a` or `b` does not have such an axis (i.e. one is lower dimensional),
2. the `i`th axis for both arrays has the same size, OR
3. at least one of `i`th axes has size $1$.

If there is some axis that is not compatible, a `Value Error` is raised.

Assuming everything is compatible, then data is "copied" (or broadcasted) to either missing axes or to inflate axes of size $1$.

#### Example of a successful broadcast

```text
a      (4d array):  8 x 1 x 6 x 1
b      (3d array):      7 x 1 x 5
Result (4d array):  8 x 7 x 6 x 5
```

In [ ]:
a = np.random.randint(10, size=(8, 1, 6, 1))
b = np.random.randint(10, size=(7, 1, 5))
result = a * b 
print(result.shape)

#### Example of an UNsuccessful broadcast

```text
a      (4d array):  8 x 1 x 6 x 1
b      (3d array):      7 x 2 x 5
Result           :   Value Error!
```

In [ ]:
# a = np.random.randint(10, size=(8, 1, 6, 1))
# b = np.random.randint(10, size=(7, 2, 5))
# result = a * b                                  # naughty naughty 

## Exercises

1. Create a $4 \times 5$ array `x` of random integers between $1$ and $99$. Use broadcasting to add $100$ to each entry to get a new array `y`. What is `y - x`, and does it make sense?
2. Create a $1$-dimensional array `a` of $6$ random (real) numbers between $3$ and $42$. Create another $1$-dimensional array `b` of $3$ random (real) numbers between $7$ and $137$.
   - What is the shape of `a * b`, or why specifically does it raise an error?
   - Reshape `a` to a $3 \times 1$ array. What is the shape of `a * b`, or why specifically does it raise an error?
   - Reshape `b` to a $2\times 3\times 1$ array. What is the shape of `a * b`, or why specifically does it raise an error?
3. Create a $3\times 2\times 4$ array `x`, a $2\times 4$ array `y`, and an array of size $4$ `z`.
   - Apply Ufuncs directly to `x`, `y`, and `z` to obtain new arrays, so that broadcasting occurs.
   - Redo the same computations with a different set of arrays so that *no broadcasting occurs*.